In [1]:
import ee

# connect to google account
#ee.Authenticate()
ee.Initialize()

In [5]:
import geemap

## prep data

# define region of interest
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -64.51171875,
              59.355596110016315
            ],
            [
              -49.92187499999999,
              63.35212928507874
            ],
            [
              -54.140625,
              71.7739410364347
            ],
            [
              -71.455078125,
              70.28911664330674
            ],
            [
              -64.51171875,
              59.355596110016315
            ]
          ]
        ]
      }
    }
  ]
}

geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -62.874755859375,
              59.70655581142613
            ],
            [
              -60.93017578124999,
              59.70655581142613
            ],
            [
              -60.93017578124999,
              60.69469537287745
            ],
            [
              -62.874755859375,
              60.69469537287745
            ],
            [
              -62.874755859375,
              59.70655581142613
            ]
          ]
        ]
      }
    }
  ]
}

# set region of interest
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# pull boundary
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                       .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                       .filter(ee.Filter.eq('instrumentMode', 'IW'))
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2021-01-11'), ee.Date('2021-01-13'))
                       .mosaic())

## create map
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
HH = ee.Image(ffa_db.select('HH'))
HV = ee.Image(ffa_db.select('HV'))

Map = geemap.Map(center=(75, -100), zoom=4)
Map.add_basemap('HYBRID')

# add layers
vis_params = {'min': -20, 'max': 0}

## add map layers
# study area
Map.addLayer(HH, vis_params, "sa_HH_D")
Map.addLayer(HV, vis_params, "sa_HV_D")

Map

Map(center=[75, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…